In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:
## Loading the model
from langchain_groq import ChatGroq
model = ChatGroq(model = 'Gemma2-9b-It', groq_api_key = groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, My name is Diwakar Tiwari and working as a backend developer")])

AIMessage(content="Hello Diwakar Tiwari,\n\nIt's nice to meet you! That's great that you're a backend developer. \n\nIs there anything specific I can help you with today? Perhaps you have a coding question, need help brainstorming ideas, or just want to chat about backend development? \n\nI'm here to assist in any way I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 24, 'total_tokens': 102, 'completion_time': 0.141818182, 'prompt_time': 0.002154675, 'queue_time': 0.233850124, 'total_time': 0.143972857}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d56ab36-623a-4a3e-b9ab-7e317f0a76d7-0', usage_metadata={'input_tokens': 24, 'output_tokens': 78, 'total_tokens': 102})

In [5]:
### Model can remember the previous messages
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello, My name is Diwakar Tiwari and working as a backend developer"),
        AIMessage(content="Hello Diwakar Tiwari!\n\nIt's nice to meet you.  \n\nBackend development is a fascinating field. What kind of technologies do you primarily work with?  \n\nI'm always eager to learn more about different aspects of software development.\n"),
        HumanMessage(content="Hey, Whats my name?")
    ]
)

AIMessage(content='Your name is Diwakar Tiwari. 😊  \n\nI remembered it from our earlier conversation! \n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 92, 'total_tokens': 126, 'completion_time': 0.061818182, 'prompt_time': 0.004268262, 'queue_time': 0.235114147, 'total_time': 0.066086444}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3edefef-82fd-4311-9b86-ab985237306d-0', usage_metadata={'input_tokens': 92, 'output_tokens': 34, 'total_tokens': 126})

In [6]:
## For particular chat there should be a chat session
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)
    

In [7]:
config = {"configurable": {"session_id":"chat1"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Diwakar Tiwari")],
    config=config
)

response.content

"Hello Diwakar Tiwari, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😄\n"

In [9]:
## After changing the session_id from chat1 to chat2, now AI don't have access of chat1 history
config1 = {"configurable": {"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello, My name is John")],
    config=config1
)

response.content

"Hello John, it's nice to meet you! 👋  What can I do for you today? 😊  \n\n"

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, what is my name?")],
    config=config1
)

response.content

'Your name is John.  I remember that you told me at the beginning of our conversation. 😊 \n\nIs there anything else I can help you with?   \n'

In [11]:
### Prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate([
    ("system","You are a helpful assistant. Answer all the question to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model

In [ ]:
chain.invoke({"messages": [HumanMessage(content="Hello, My name is Diwakar")]}) ## This will be placed inside the MessagePlaceholder

AIMessage(content="Hello Diwakar, it's nice to meet you!  \n\nWhat can I help you with today? I'm ready for any questions you have. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 34, 'total_tokens': 73, 'completion_time': 0.070909091, 'prompt_time': 0.003372993, 'queue_time': 0.233393496, 'total_time': 0.074282084}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24c70066-3404-4da0-a2e0-04b35ad5e0a3-0', usage_metadata={'input_tokens': 34, 'output_tokens': 39, 'total_tokens': 73})

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [15]:
config = {"configurable": {"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Diwakar")],
    config=config
)

response.content

"Hello Diwakar, it's nice to meet you! How can I help you today? 😊  \n\nPlease ask away, I'm ready for your questions.  \n\n"

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
    ("system",
     "You are a helpful assistant. Answer all the question to the best of your ability in {language}."
    ),
    MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke({"messages":[HumanMessage(content="My name is Diwakar")], "language":"Hindi"})
response.content

'नमस्ते Diwakar! \n\nमुझे बहुत खुशी है कि आप मेरे साथ बात कर रहे हैं। आप जो भी सवाल पूछेंगे, मैं अपनी पूरी कोशिश करूंगा कि आपको सबसे अच्छे जवाब दें। \n\nआपका क्या सवाल है? \n\n'

In [19]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="I am Diwakar")], "language":"Hindi"},
    config=config
)

response.content

'नमस्ते दीवाकर!  \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप जो भी प्रश्न पूछेंगे, मैं उसे अपनी पूरी क्षमता के अनुसार उत्तर देने की कोशिश करूंगा। \n\nआप किस बारे में जानना चाहते हैं? \n\n'

In [21]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hey, What is my name?")], "language":"Hindi"},
    config=config
)

response.content

'आपका नाम दीवाकर है!  \n'

In [ ]:
### Managing the Conversation History